In [6]:
# Imports

from gedcom.element.individual import IndividualElement
from gedcom.element.element import Element
from gedcom.parser import Parser

# Théoph
from ipynb.fs.full.explorertheoph import *

# Alice
from ipynb.fs.full.explorer import *

# Alison
# from exploreralison import *

In [10]:
file_path = 'Queen_Eliz_II.ged'
gedcom_parser = Parser()
gedcom_parser.parse_file(file_path)
root_child_elements = gedcom_parser.get_root_child_elements()

In [55]:
def get_name_str(Individual) :
    (first, last) = Individual.get_name()
    name = first + " " + last
    return name

In [4]:
def print_family(Individual) :
    """
    Prints names of all members of Individual's family

    Parameters
    ---
    Individual : IndividualElement

    Returns
    ---
    Nothing
    """

    name_ind = get_name_str(Individual) # get_name_str à écrire
    name_spouse = get_name_str(get_spouse(Individual)) 
    if Individual.get_gender() == "M" :
        parents = [name_ind, name_spouse]
    else : 
        parents = [name_spouse, name_ind]
    print("Parents : " + parents[0] + ", " + parents[1] +"\n")

    children_id = get_children(Individual)
    children = []
    print("Children : ")
    for i in range(len(children_id)-1) :
        print(get_name_str(children_id[i]) + ", ")
    print(get_name_str(children_id[-1]))

In [57]:
Individual = find_IndividualElement("De Brus Robert Bruce", "de Brusse")[1]
# gedcom_parser.get_families(Individual,"FAMS")

# Trouve la famille dont l'individu est un des parents
gedcom_parser.get_families(Individual)

# Trouve la famille dont l'individu est enfant
gedcom_parser.get_families(Individual, "gedcom_parser.tags.GEDCOM_TAG_FAMILY_CHILD")

# Nombre de familles par individu 
liste_individus = gedcom_parser.get_root_child_elements()
list_spouse = [0,0,0,0]
list_child = [0,0,0,0]
for x in liste_individus :
    if x.get_tag() == "INDI" :
        nb_spouse = len(gedcom_parser.get_families(x))
        nb_child = len(gedcom_parser.get_families(x,"gedcom_parser.tags.GEDCOM_TAG_FAMILY_CHILD"))
        if nb_spouse < 4 :
            list_spouse[nb_spouse] += 1
        if nb_child < 4 :
            list_child[nb_child] += 1
print("Nombre d'individus marriés : ", list_spouse)
print("Nombre d'individus enfants d'une famille : ", list_child)

# Trouve les parents d'un individu
parents = gedcom_parser.get_parents(Individual)
print(get_name_str(parents[0]), " and ", get_name_str(parents[1]))



Nombre d'individus marriés :  [0, 4624, 55, 3]
Nombre d'individus enfants d'une famille :  [4683, 0, 0, 0]
Ragnvald (Rognvald) II Brusesson [Earl Of Orkney]   and  Arlogia Countess Of Orkney 
